<a href="https://colab.research.google.com/github/hancb8539/C-sharp/blob/main/q_learning_maze_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pds
import random
import copy
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from collections import deque
from keras import backend as K

class Maze(object):
    def __init__(self, size=10, blocks_rate=0.2):
        self.size = size if size > 3 else 10
        self.blocks = int((size ** 2) * blocks_rate) 
        self.s_list = []
        self.maze_list = []
        self.e_list = []

    def create_mid_lines(self, k):
        if k == 0: self.maze_list.append(self.s_list)
        elif k == self.size - 1: self.maze_list.append(self.e_list)
        else:
            tmp_list = []
            for l in range(0,self.size):
                if l == 0: tmp_list.extend("#")
                elif l == self.size-1: tmp_list.extend("#")
                else:
                    #a = random.randint(-1, 0)
                    a = 0
                    tmp_list.extend([a])
            self.maze_list.append(tmp_list)

    def insert_blocks(self, k, s_r, e_r):
        b_y = random.randint(1, self.size-2)
        b_x = random.randint(1, self.size-2)
        if [b_y, b_x] == [1, s_r] or [b_y, b_x] == [self.size - 2, e_r]: k = k-1
        else: self.maze_list[b_y][b_x] = "#"
            
    def generate_maze(self): 
        s_r = random.randint(1, (self.size / 2) - 1)
        for i in range(0, self.size):
            if i == s_r: self.s_list.extend("S")
            else: self.s_list.extend("#")
        start_point = [0, s_r]

        e_r = random.randint((self.size / 2) + 1, self.size - 2)
        for j in range(0, self.size):
            if j == e_r: self.e_list.extend([50])
            else: self.e_list.extend("#")
        goal_point = [self.size - 1, e_r]

        for k in range(0, self.size):
            self.create_mid_lines(k)
        
        for k in range(self.blocks):
            self.insert_blocks(k, s_r, e_r)

        return self.maze_list, start_point, goal_point

class Field(object):
    def __init__(self, maze, start_point, goal_point):
        self.maze = maze
        self.start_point = start_point
        self.goal_point = goal_point
        self.movable_vec = [[1,0],[-1,0],[0,1],[0,-1],[1,-1],[1,1],[-1,1],[-1,-1]]

    def display(self, point=None):
        field_data = copy.deepcopy(self.maze)
        if not point is None:
                y, x = point
                field_data[y][x] = "@@"
        else:
                point = ""
        for line in field_data:
                print ("\t" + "%3s " * len(line) % tuple(line))

    def get_actions(self, state):
        movables = []
        if state == self.start_point:
            y = state[0] + 1
            x = state[1]
            a = [[y, x]]
            return a
        else:
            for v in self.movable_vec:
                y = state[0] + v[0]
                x = state[1] + v[1]
                if not(0 < x < len(self.maze) and
                       0 <= y <= len(self.maze) - 1 and
                       maze[y][x] != "#" and
                       maze[y][x] != "S"):
                    continue
                movables.append([y,x])
            if len(movables) != 0:
                return movables
            else:
                return None

    def get_val(self, state):
        y, x = state
        if state == self.start_point: return 0, False
        else:
            v = float(self.maze[y][x])
            if state == self.goal_point: 
                return v, True
            else: 
                return v, False

size = 20
barriar_rate = 0.1

maze_1 = Maze(size, barriar_rate)
maze, start_point, goal_point = maze_1.generate_maze()
maze_field = Field(maze, start_point, goal_point)

maze_field.display()

class Node(object):    
    def __init__(self, state, start_point, goal_point):
        self.state = state
        self.start_point = start_point
        self.goal_point = goal_point
        self.hs = (self.state[0] - self.goal_point[0]) ** 2 + (self.state[1] - self.goal_point[1]) ** 2
        self.fs = 0
        self.parent_node = None
    
    def confirm_goal(self):
        if self.goal_point == self.state: return True
        else: return False

class NodeList(list):
    def find_nodelist(self, state):
        node_list = [t for t in self if t.state==state]
        return node_list[0] if node_list != [] else None
    def remove_from_nodelist(self, node):
        del self[self.index(node)]

class DQN_Solver:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)
        self.gamma = 0.9
        self.epsilon = 1.0
        self.e_decay = 0.9999
        self.e_min = 0.01
        self.learning_rate = 0.0001
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(128, input_shape=(2,2), activation='tanh'))
        model.add(Flatten())
        model.add(Dense(128, activation='tanh'))
        model.add(Dense(128, activation='tanh'))
        model.add(Dense(1, activation='linear'))
        model.compile(loss="mse", optimizer=RMSprop(lr=self.learning_rate))
        return model

    def remember_memory(self, state, action, reward, next_state, next_movables, done):
        self.memory.append((state, action, reward, next_state, next_movables, done))

    def choose_action(self, state, movables):
        if self.epsilon >= random.random():
            return random.choice(movables)
        else:
            return self.choose_best_action(state, movables)
        
    def choose_best_action(self, state, movables):
        best_actions = []
        max_act_value = -100
        for a in movables:
            np_action = np.array([[state, a]])
            act_value = self.model.predict(np_action)
            if act_value > max_act_value:
                best_actions = [a,]
                max_act_value = act_value
            elif act_value == max_act_value:
                best_actions.append(a)
        return random.choice(best_actions)

    def replay_experience(self, batch_size):
        batch_size = min(batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)
        X = []
        Y = []
        for i in range(batch_size):
            state, action, reward, next_state, next_movables, done = minibatch[i]
            input_action = [state, action]
            if done:
                target_f = reward
            else:
                next_rewards = []
                for i in next_movables:
                    np_next_s_a = np.array([[next_state, i]])
                    next_rewards.append(self.model.predict(np_next_s_a))
                np_n_r_max = np.amax(np.array(next_rewards))
                target_f = reward + self.gamma * np_n_r_max
            X.append(input_action)
            Y.append(target_f)
        np_X = np.array(X)
        np_Y = np.array([Y]).T
        self.model.fit(np_X, np_Y, epochs=1, verbose=0)
        if self.epsilon > self.e_min:
            self.epsilon *= self.e_decay

state_size = 2
action_size = 2
dql_solver = DQN_Solver(state_size, action_size)

episodes = 500
times = 1000

for e in range(episodes):
    state = start_point
    score = 0
    for time in range(times):
        movables = maze_field.get_actions(state)
        action = dql_solver.choose_action(state, movables)
        reward, done = maze_field.get_val(action)
        score = score + reward
        next_state = action
        next_movables = maze_field.get_actions(next_state)
        dql_solver.remember_memory(state, action, reward, next_state, next_movables, done)
        if done or time == (times - 1):
            if e % 500 == 0:
                print("episode: {}/{}, score: {}, e: {:.2} \t @ {}"
                        .format(e, episodes, score, dql_solver.epsilon, time))
            break
        state = next_state
    dql_solver.replay_experience(32)

state = start_point
score = 0
steps = 0
while True:
    steps += 1
    movables = maze_field.get_actions(state)
    action = dql_solver.choose_best_action(state, movables)
    print("current state: {0} -> action: {1} ".format(state, action))
    reward, done = maze_field.get_val(action)
    maze_field.display(state)
    score = score + reward
    state = action
    print("current step: {0} \t score: {1}\n".format(steps, score))
    if done:
        maze_field.display(action)
        print("goal!")
        break



	  #   #   #   #   #   #   #   #   S   #   #   #   #   #   #   #   #   #   #   # 
	  #   0   0   #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   # 
	  #   0   0   #   #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   # 
	  #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   # 
	  #   0   0   0   0   0   #   0   0   #   0   0   0   0   0   #   0   0   0   # 
	  #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   #   0   0   # 
	  #   0   0   #   0   0   #   0   0   0   0   0   #   0   0   0   0   0   0   # 
	  #   0   0   0   #   0   0   0   #   #   0   0   0   0   0   0   #   0   0   # 
	  #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   # 
	  #   0   0   0   0   0   0   0   #   #   #   0   0   0   0   0   0   0   0   # 
	  #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   #   # 
	  #   0   0   0   0   0   0   0   0   0   #   #   0   0   0   0   0   #   0   # 
	  #   0   0   0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


current state: [0, 8] -> action: [1, 8] 
	  #   #   #   #   #   #   #   #  @@   #   #   #   #   #   #   #   #   #   #   # 
	  #   0   0   #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   # 
	  #   0   0   #   #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   # 
	  #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   # 
	  #   0   0   0   0   0   #   0   0   #   0   0   0   0   0   #   0   0   0   # 
	  #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   #   0   0   # 
	  #   0   0   #   0   0   #   0   0   0   0   0   #   0   0   0   0   0   0   # 
	  #   0   0   0   #   0   0   0   #   #   0   0   0   0   0   0   #   0   0   # 
	  #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   # 
	  #   0   0   0   0   0   0   0   #   #   #   0   0   0   0   0   0   0   0   # 
	  #   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   #   # 
	  #   0   0   0   0   0   0   0   0   0   #   #   0   0 

In [ ]:
import tensorflow as tf
import numpy as np
from collections import deque
import random
import datetime

tf.compat.v1.disable_eager_execution()
class DeepQNetwork:
    r_list = np.array([[-100,0,-100,-100,-100,0,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100],
[0,-100,0,-100,-100,-100,0,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100],
[-100,0,-100,100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100],
[-100,-100,0,100,0,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100],
[-100,-100,-100,100,-100,-100,-100,-100,-100,0,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100],
[0,-100,-100,-100,-100,-100,0,-100,-100,-100,0,-100,-100,-100,-100,-100,-100,-100,-100,-100],
[-100,0,-100,-100,-100,0,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100],
[-100,-100,0,-100,-100,-100,0,-100,0,-100,-100,-100,0,-100,-100,-100,-100,-100,-100,-100],
[-100,-100,-100,100,-100,-100,-100,-100,-100,0,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100],
[-100,-100,-100,-100,0,-100,-100,-100,0,-100,-100,-100,-100,-100,0,-100,-100,-100,-100,-100],
[-100,-100,-100,-100,-100,0,-100,-100,-100,-100,-100,-100,-100,-100,-100,0,-100,-100,-100,-100],
[-100,-100,-100,-100,-100,-100,0,-100,-100,-100,0,-100,0,-100,-100,-100,0,-100,-100,-100],
[-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,0,-100,-100],
[-100,-100,-100,-100,-100,-100,-100,-100,0,-100,-100,-100,0,-100,0,-100,-100,-100,0,-100],
[-100,-100,-100,-100,-100,-100,-100,-100,-100,0,-100,-100,-100,-100,-100,-100,-100,-100,-100,0],
[-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,0,-100,-100,-100,-100,-100,0,-100,-100,-100],
[-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,0,-100,0,-100,-100],
[-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,0,-100,-100,-100,0,-100,0,-100],
[-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,0,-100,0],
[-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,-100,0,-100,-100,-100,0,-100]])

    
    def __init__(self):
        self.learning_rate = 0.001  #神经网络的学习速率设为0.001
        self.state_num = 20  #有20个位置状态
        self.action_num = 20  #有20个动作
        self.epsilon = 0.9   #动作选择概率初始值设为0.9
        self.epsilon_final = 0.9999 
        self.state_list = np.identity(self.state_num)  
        self.action_list = np.identity(self.action_num)  
        self.relay_memory_store = deque()  
        self.memory_size = 10000  
        self.observe = 2500  
        self.batch_mini = 200  
        self.gamma = 0.9  
        self.learn_step_counter = 0 
        self.train_step_counter = 0   
        self.bug = 0  
        self.creat_network()
        self.session = tf.compat.v1.InteractiveSession()
        self.session.run(tf.compat.v1.global_variables_initializer())

    def creat_network(self):
        self.q_eval_input = tf.compat.v1.placeholder(shape=[None,self.state_num],dtype=tf.float32)   
        self.action_input = tf.compat.v1.placeholder(shape=[None,self.action_num],dtype=tf.float32)  
        self.q_target = tf.compat.v1.placeholder(shape=[None],dtype=tf.float32) 
        neuro_layer_1 = 8  #隐含层
        w1 = tf.Variable(tf.compat.v1.random_normal([self.state_num,neuro_layer_1]))  
        b1 = tf.Variable(tf.zeros([1,neuro_layer_1])+0.1)  
        l1 = tf.nn.relu(tf.matmul(self.q_eval_input,w1)+b1)  
        w2 = tf.Variable(tf.compat.v1.random_normal([neuro_layer_1,self.state_num]))  
        b2 = tf.Variable(tf.zeros([1,self.action_num])+0.1)  
        self.q_eval = tf.matmul(l1,w2)+b2   
        self.reward_action = tf.compat.v1.reduce_sum(tf.multiply(self.q_eval,self.action_input),reduction_indices=1)  
        self.loss = tf.compat.v1.reduce_mean(tf.square(self.q_target - self.reward_action))   
        self.train_op = tf.compat.v1.train.GradientDescentOptimizer(self.learning_rate).minimize(self.loss) 
        

        
    def choose_action(self,state_index):
        current_state = self.state_list[state_index:state_index + 1]  
        if ((np.random.uniform() > self.epsilon ) or  (self.train_step_counter < self.observe)):  
            action_index = np.random.randint(0,self.action_num) 
        else:
            action_q = self.session.run(self.q_eval,feed_dict={self.q_eval_input:current_state})  
            action_index = np.argmax(action_q) 
        return action_index  
    

    def save(self,state0,action0,reward0,next_state0,done0): 
        current_state1 =  self.state_list[state0:state0+1]
        current_action1 = self.action_list[action0:action0+1]
        next_state1 = self.state_list[next_state0:next_state0+1]
        self.relay_memory_store.append((current_state1,current_action1,reward0,next_state1,done0))  #保存
        if len(self.relay_memory_store) > self.memory_size:  
            self.relay_memory_store.popleft()  


    def experience_replay(self):   #经验回放
        batch = random.sample(self.relay_memory_store,self.batch_mini)  
        batch_state = None
        batch_action = None
        batch_reward = None
        batch_next_state = None
        batch_done = None
        for i in range(self.batch_mini):  
            batch_state = batch[i][0] if batch_state is None else np.vstack((batch_state,batch[i][0])) 
            batch_action = batch[i][1] if batch_action is None else np.vstack((batch_action,batch[i][1]))
            batch_reward = batch[i][2] if batch_reward is None else np.vstack((batch_reward,batch[i][2]))
            batch_next_state = batch[i][3] if batch_next_state is None else np.vstack((batch_next_state,batch[i][3]))
            batch_done = batch[i][4] if batch_done is None else np.vstack((batch_done,batch[i][4]))
        q_target = []
        q_next = self.session.run(self.q_eval,feed_dict={self.q_eval_input:batch_next_state})
        for i in range(self.batch_mini):
            R = batch_reward[i][0]  
            q_value = R + self.gamma * np.max(q_next[i]) 
            if R < 0:
                q_target.append(R)
            else:
                q_target.append(q_value)  
        loss3 = self.session.run([self.train_op,self.loss],feed_dict={self.q_eval_input:batch_state,self.action_input:batch_action,self.q_target:q_target})  
        self.learn_step_counter += 1  

        
    def train(self):
        current_state = np.random.randint(0,self.state_num)  
        while True:
            current_action = self.choose_action(current_state)  
            next_state = current_action 
            current_reward = self.r_list[current_state][current_action]  
            done = True  if current_state == 3 else False  
            self.save(current_state,current_action,current_reward,next_state,done)  
            if self.train_step_counter > self.observe:  
                self.experience_replay()  
            if self.train_step_counter > 40000:   
                break
            if done:
                current_state = np.random.randint(0,self.state_num)        
            else:
                current_state = next_state
            if self.train_step_counter > self.observe and self.epsilon < self.epsilon_final:  
                self.epsilon += 0.00001
            self.train_step_counter += 1
            if self.train_step_counter % 1000 ==0:  
                print(self.train_step_counter)
                

    def play(self):
        print('---开始训练---')
        self.train()
        print('---训练结束---')
        for i in range(20):  
            if self.bug >10:  
                break
            else:
                self.bug = 0
            print('\n第',i,'轮')
            start_state = i  
            print('从',start_state,'位置出发')
            current_state = start_state
            while current_state != 3:  
                out_result = self.session.run(self.q_eval,feed_dict={self.q_eval_input:self.state_list[current_state:current_state+1]})
                out_next_state = np.argmax(out_result[0])
                if out_next_state != 3:
                    print('经过',out_next_state,'位置')
                else:
                    print('到达终点3')
                current_state = out_next_state
                if self.bug > 10:  
                    print('-------神经网络训练不好------')
                    break
                else:
                    self.bug +=1
                
    
if __name__ == "__main__":
    start_time = datetime.datetime.now()
    q_network = DeepQNetwork()
    q_network.play()
    end_time = datetime.datetime.now()
    print('\n----------用时',int((end_time - start_time).seconds/60),'分',(end_time - start_time).seconds%60,'秒-----------')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1766: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


串流輸出內容已截斷至最後 5000 行。
current_action: 1
current_reward: -100
current_action: 9
current_reward: -100
current_action: 7
current_reward: -100
current_action: 1
current_reward: -100
current_action: 17
current_reward: -100
current_action: 5
current_reward: -100
current_action: 9
current_reward: -100
current_action: 5
current_reward: -100
current_action: 5
current_reward: -100
current_action: 3
current_reward: -100
current_action: 13
current_reward: -100
current_action: 2
current_reward: -100
current_action: 3
current_reward: 100
current_action: 6
current_reward: -100
current_action: 9
current_reward: -100
current_action: 5
current_reward: -100
current_action: 0
current_reward: 0
current_action: 13
current_reward: -100
current_action: 8
current_reward: 0
current_action: 8
current_reward: -100
current_action: 19
current_reward: -100
current_action: 19
current_reward: -100
current_action: 1
current_reward: -100
current_action: 11
current_reward: -100
current_action: 2
current_reward: -100
curre

KeyboardInterrupt: ignored

In [2]:
from google.colab import files
# 上傳csv檔案
uploaded = files.upload()

Saving LM386 Amplifier.dsn to LM386 Amplifier.dsn


In [ ]:
# maze.py
import io
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from tqdm import trange, tqdm
import time

time_start=time.time()
pcbline = [[0,2116/8,2116/8,0,0],[0,0,1360/8,1360/8,0]]
environment_x = int(2116/8)
environment_y = int(1360/8)

actions = ['up', 'down', 'left', 'right', 'lefttop', 'leftdown', 'righttop', 'rightdown']
# =============================================================
pcb = []
component = []
pin = []
pin_dir = {} #pin_name轉成字典
padstack = []
net = []
result = []
maze = []
F = np.full((environment_x,environment_y,2),1)  # Feasible 1可走 0不可走
class Comp():
    def __init__(self, component_name=None, instance_name=None, x=0, y=0, side=None, angle=0):
        self.component_name = component_name
        self.instance_name = instance_name
        self.x = x
        self.y = y
        self.side = side
        self.angle = angle

def read():
    data = open('LM386 Amplifier.dsn', 'r')
    lines = data.readline()
    lines = lines.strip()
    c = c1 =  c2 = c3 = -1
    while lines:
        c = c + 1
        pcb.append([])
        tmp = lines.split(' ')
        for i in range(len(tmp)):
            pcb[c].append(tmp[i])
        
        lines = data.readline()
        lines = lines.strip()
    count = 0
    for i in range(len(pcb)-1):
        if pcb[i][0] == '(component':
            component_name = pcb[i][1]
            if pcb[i+1][0] == '(place':
                angle = pcb[i+1][5].replace('))','')
                component.append(Comp(component_name,pcb[i+1][1],float(pcb[i+1][2]) / 8,float(pcb[i+1][3]) / 8,pcb[i+1][4],int(angle)))
        if pcb[i][0] == '(image':
            c1 = c1 + 1
            pin.append([])
            pin[c1].append(pcb[i][1])
        elif pcb[i][0] == '(pin':
            pin[c1].append(pcb[i])
            for j in range(len(pin[c1])):
                if j != 0:
                    pin[c1][j][3] = float(pin[c1][j][3])
                    pin[c1][j][4] = float(pin[c1][j][4])
                    pin[c1][j][3] = int(pin[c1][j][3])
                    pin[c1][j][4] = int(pin[c1][j][4])
        if pcb[i][0] == '(padstack':
            c2 = c2 + 1
            padstack.append([])
            padstack[c2].append(pcb[i][1])
            if pcb[i+1][0] == '(shape':
                for j in range(len(pcb[i+1])):
                    padstack[c2].append(pcb[i+1][j])
            if(padstack[c2][2] == '(circle'):
                padstack[c2][4] = padstack[c2][4].replace('))','')
                padstack[c2][4] = float(padstack[c2][4])
                padstack[c2][4] = int(padstack[c2][4])
            if(padstack[c2][2] == '(rect'):
                padstack[c2][4] = float(padstack[c2][4])
                padstack[c2][5] = float(padstack[c2][5])
                padstack[c2][6] = float(padstack[c2][6])
                padstack[c2][4] = int(padstack[c2][4]/8)
                padstack[c2][5] = int(padstack[c2][5]/8)
                padstack[c2][6] = int(padstack[c2][6]/8)
                padstack[c2][7] = padstack[c2][7].replace('))','')
                padstack[c2][7] = float(padstack[c2][7])
                padstack[c2][7] = int(padstack[c2][7]/8)
        if pcb[i][0] == '(net':
            c3 = c3 + 1
            net.append([])
            net[c3].append(pcb[i][1])
            count = count + 1
            continue
        if pcb[i][0] == '(pins':
            count = count + 1
            continue
        if (count == 2) and (pcb[i][0] != ')'):
            for j in range(len(pcb[i])):
                net[c3].append(pcb[i][j])
        if (count == 2) and (pcb[i][0] == ')'):
            count = 0 
    for j in range(len(component)):
        component[j].x = int(component[j].x) - 187
        component[j].y = int(component[j].y) - 187
        component[j].x = int(component[j].x)
        component[j].y = int(component[j].y)
    for i in range(len(pin)):
        for j in range(len(pin[i])):
            if j == 0: continue
            pin[i][j][3] = int(pin[i][j][3]/8)
            pin[i][j][4] = int(pin[i][j][4]/8)
    for i in range(len(pin)):
        for j in range(len(pin[i])):
            if j == 0: continue
            for k in range(len(component)):
                if component[k].component_name == pin[i][0]:
                    pin[i][j][3] = pin[i][j][3] + component[k].x
                    pin[i][j][4] = pin[i][j][4] + component[k].y
                    name = component[k].instance_name + '-' + pin[i][j][2]
                    pin[i][j][2] = name
                    pin_dir[len(pin_dir)] = name
    print(pin)
# =============================================================
def my_print(Q):
    # hard-coded hack for this problem only
    rows = len(Q); cols = len(Q[0])
    print("       0      1      2      3      4      5      6      7")
    for i in range(rows):
        print("%d " % i, end="")
        if i < 10: print(" ", end="")
        for j in range(cols): print(" %6.2f" % Q[i,j], end="")
        print("")
    print("")

def get_shortest_path(s, poss_next_states, trans, goal):
    small = 9999.9
    idx = -1
    for i in range(len(poss_next_states)):
        if poss_next_states[i] == 0:
            dist = math.sqrt((trans[s][0] - trans[goal][0]) ** 2 + (trans[s][1]+1 - trans[goal][1]) ** 2)
            if dist < small: 
                small = dist
                idx = poss_next_states[i]
        elif poss_next_states[i] == 1:
            dist = math.sqrt((trans[s][0] - trans[goal][0]) ** 2 + (trans[s][1]-1 - trans[goal][1]) ** 2)
            if dist < small: 
                small = dist
                idx = poss_next_states[i]
        elif poss_next_states[i] == 2:
            dist = math.sqrt((trans[s][0]-1 - trans[goal][0]) ** 2 + (trans[s][1] - trans[goal][1]) ** 2)
            if dist < small: 
                small = dist
                idx = poss_next_states[i]
        elif poss_next_states[i] == 3:
            dist = math.sqrt((trans[s][0]+1 - trans[goal][0]) ** 2 + (trans[s][1] - trans[goal][1]) ** 2)
            if dist < small: 
                small = dist
                idx = poss_next_states[i]
        elif poss_next_states[i] == 4:
            dist = math.sqrt((trans[s][0]-1 - trans[goal][0]) ** 2 + (trans[s][1]+1 - trans[goal][1]) ** 2)
            if dist < small: 
                small = dist
                idx = poss_next_states[i]
        elif poss_next_states[i] == 5:
            dist = math.sqrt((trans[s][0]-1 - trans[goal][0]) ** 2 + (trans[s][1]-1 - trans[goal][1]) ** 2)
            if dist < small: 
                small = dist
                idx = poss_next_states[i]
        elif poss_next_states[i] == 6:
            dist = math.sqrt((trans[s][0]+1 - trans[goal][0]) ** 2 + (trans[s][1]+1 - trans[goal][1]) ** 2)
            if dist < small: 
                small = dist
                idx = poss_next_states[i]
        elif poss_next_states[i] == 7:
            dist = math.sqrt((trans[s][0]+1 - trans[goal][0]) ** 2 + (trans[s][1]-1 - trans[goal][1]) ** 2)
            if dist < small: 
                small = dist
                idx = poss_next_states[i]
    return idx    
    
def get_poss_next_states(s, maze, trans):
    # given a state s and a feasibility matrix F
    # get list of possible next states
    poss_next_states = []
    #print("ss: ",s)
    for j in range(0,8):
        if maze[s,j] == 1: 
            poss_next_states.append(j)
    if not poss_next_states:
      return None
    return poss_next_states

def get_rnd_next_state(s, maze, trans, key_val, lrn_rate, Q, goal, count):
    # given a state s, pick a feasible next state
    #print("s: ",s)
    poss_next_states = get_poss_next_states(s, maze, trans)
    if poss_next_states == None:
      return None
    #print(poss_next_states)
    if np.random.uniform() < lrn_rate:
        next_state = poss_next_states[np.random.randint(0,len(poss_next_states))]
        #print(next_state)
    else:
        if count < 100:
            next_state = get_shortest_path(s, poss_next_states, trans, goal)
        else: 
            next_state = poss_next_states[np.random.randint(0,len(poss_next_states))]
    #next_state = poss_next_states[np.random.randint(0,len(poss_next_states))]
    for i in range(len(key_val[s])):
        if key_val[s][i][0] == next_state:
            #print("key_val: ",key_val[s])
            return key_val[s][i][1]
        
# =============================================================

def train(maze, R, Q, gamma, lrn_rate, start, goal, trans, max_epochs, key_val):
    # compute the Q matrix
    trans_len = len(trans)
    for i in range(0,max_epochs):
        print(i)
        count = 0
        train_time_start=time.time() 
        curr_s = np.random.randint(0,trans_len)
        while(True): #nett = [[(223,106),(217,51)]]
            if trans[start][0] > trans[goal][0] and trans[start][1] > trans[goal][1]:
                if trans[curr_s][0] > trans[start][0] or trans[curr_s][1] > trans[start][1] or trans[curr_s][0] < trans[goal][0] or trans[curr_s][1] < trans[goal][1]:
                    #print("trans[curr_s]:",trans[curr_s][0]," ",trans[curr_s][1])
                    curr_s = np.random.randint(0,trans_len)
                else: break
            elif trans[start][0] > trans[goal][0] and trans[start][1] < trans[goal][1]:
                if trans[curr_s][0] > trans[start][0] or trans[curr_s][1] < trans[start][1] or trans[curr_s][0] < trans[goal][0] or trans[curr_s][1] > trans[goal][1]:
                    #print("trans[curr_s]:",trans[curr_s][0]," ",trans[curr_s][1])
                    curr_s = np.random.randint(0,trans_len)
                else: break
            elif trans[start][0] < trans[goal][0] and trans[start][1] > trans[goal][1]:
                if trans[curr_s][0] < trans[start][0] or trans[curr_s][1] > trans[start][1] or trans[curr_s][0] > trans[goal][0] or trans[curr_s][1] < trans[goal][1]:
                    #print("trans[curr_s]:",trans[curr_s][0]," ",trans[curr_s][1])
                    curr_s = np.random.randint(0,trans_len)
                else: break
            elif trans[start][0] < trans[goal][0] and trans[start][1] < trans[goal][1]:
                if trans[curr_s][0] < trans[start][0] or trans[curr_s][1] < trans[start][1] or trans[curr_s][0] > trans[goal][0] or trans[curr_s][1] > trans[goal][1]:
                    #print("trans[curr_s]:",trans[curr_s][0]," ",trans[curr_s][1])
                    curr_s = np.random.randint(0,trans_len)
                else: break
            elif trans[start][0] == trans[goal][0] and trans[start][1] > trans[goal][1]: #(96 , 110  to  96 , 86)
                if trans[curr_s][0] > trans[start][0]+30 or trans[curr_s][1] > trans[start][1] or trans[curr_s][0] < trans[goal][0]-30 or trans[curr_s][1] < trans[goal][1]:
                    #print("trans[curr_s]:",trans[curr_s][0]," ",trans[curr_s][1])
                    curr_s = np.random.randint(0,trans_len)
                else: break
            elif trans[start][0] == trans[goal][0] and trans[start][1] < trans[goal][1]:
                if trans[curr_s][0] > trans[start][0]+30 or trans[curr_s][1] < trans[start][1] or trans[curr_s][0] < trans[goal][0]-30 or trans[curr_s][1] > trans[goal][1]:
                    curr_s = np.random.randint(0,trans_len)
                else: break
            elif trans[start][0] > trans[goal][0] and trans[start][1] == trans[goal][1]:
                if trans[curr_s][0] > trans[start][0] or trans[curr_s][1] > trans[start][1]+30 or trans[curr_s][0] < trans[goal][0] or trans[curr_s][1] < trans[goal][1]-30:
                    curr_s = np.random.randint(0,trans_len)
                else: break
            elif trans[start][0] < trans[goal][0] and trans[start][1] == trans[goal][1]:
                if trans[curr_s][0] < trans[start][0] or trans[curr_s][1] > trans[start][1]+30 or trans[curr_s][0] > trans[goal][0] or trans[curr_s][1] < trans[goal][1]-30:
                    curr_s = np.random.randint(0,trans_len)
                else: break
  
        while(True):
            count = count + 1
            next_s = get_rnd_next_state(curr_s, maze, trans, key_val, lrn_rate, Q, goal, count)
            #print("next_s ",next_s)
            if next_s == None:
              break
            poss_next_next = get_poss_next_states(next_s, maze, trans)
            if poss_next_next == None:
              break
            #print("poss_next_next ",poss_next_next)
            poss_next_next_states = []
            for k in range(len(key_val[next_s])):
                for j in range(len(poss_next_next)):
                        if key_val[next_s][k][0] == poss_next_next[j]:
                            poss_next_next_states.append(key_val[next_s][k][1])
            #print("poss_next_next_states ",poss_next_next_states)
            max_Q = -9999.99
            q = -1
            for j in range(len(poss_next_next_states)):
                nn_s = poss_next_next_states[j]
                #print("nn_s ",nn_s)
                for k in range(len(key_val[next_s])):
                    if key_val[next_s][k][1] == nn_s:
                        q = Q[next_s,key_val[next_s][k][0]]
                        #print("next_s,k ",next_s," ",key_val[k][1]," ",q)
                        break
                if q > max_Q:
                    max_Q = q
            for k in range(len(key_val[curr_s])):
                if key_val[curr_s][k][1] == next_s:
                    Q[curr_s][key_val[curr_s][k][0]] = ((1 - lrn_rate) * Q[curr_s][key_val[curr_s][k][0]]) + (lrn_rate * (R[curr_s][key_val[curr_s][k][0]] + (gamma * max_Q)))
                    curr_s = next_s
                    break
            if curr_s == goal:
                break
        train_time_end=time.time()
# =============================================================

def walk(start, goal, Q, trans, key_val):
    # go to goal from start using Q
    pathx = []
    pathy = []
    curr = start
    curr_pos = trans.get(curr)
    goal_pos = trans.get(goal)
    print(str(curr_pos) + "->", end="")
    pathx.append(curr_pos[0])
    pathy.append(curr_pos[1])
    count = 0
    while curr != goal:
        count = count + 1
        #print(Q[curr])
        next_index = np.argmax(Q[curr])
        #Q[curr,next_index] = -1
        #print("next_index ",next_index)
        next = -1
        for i in range(len(key_val[curr])):
            if key_val[curr][i][0] == next_index:
                next = key_val[curr][i][1]
                #print("next ",next)
                break
        next_pos = trans.get(next)
        if next_pos != None: 
            pathx.append(next_pos[0])
            pathy.append(next_pos[1])
            print(str(next_pos) + "->", end="")
            print("Q value: ",Q[next])
            curr = next
        if count == 100000: 
            print("None")
            break
    print("done")
    if pathx[len(pathx)-1] == goal_pos[0] and pathy[len(pathy)-1] == goal_pos[1]:
        result.append([])
        result[len(result)-1].append(line_to_line(pathx,pathy))
        print("path: ",result[len(result)-1])
        draw(result)
        return pathx,pathy
    else: return None,None
        
# =============================================================
def get_key(trans,value):
    for k,v in trans.items():
        if v == value:
            return k
# =============================================================
def point_connect(dir):
    l = len(dir)
    if l <= 1:
        return [], 0
 
    con = [dir[0]]     # 已經連線的點集，先隨便放一个點進去
    not_con = dir[1:]  # 還沒連線的點集
    paths = []          # 所有連線
    for _ in range(l - 1):  # 共 l-1 條連線
        # 得到下一條連線的兩點a、b 及其距離length_ab
        a, b = con[0], not_con[0]  # 先任意選兩個點
        length_ab = math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)
        for m in con:
            for n in not_con:
                lg = math.sqrt((m[0] - n[0]) ** 2 + (m[1] - n[1]) ** 2)
                if lg < length_ab:  # 如果有更短的
                    length_ab = lg
                    a, b = m, n
 
        # 記錄
        paths.append([dir.index(a), dir.index(b)])   # 記錄連線ab
        con.append(b)      # 已連接點集中記錄點b
        not_con.remove(b)  # 未連接點集中删除點b
 
    return paths

# =============================================================
def draw(result):  
    plt.rcParams['savefig.dpi'] = 150
    plt.rcParams['figure.dpi'] = 150
    topx = []
    topy = []
    plt.plot(pcbline[0], pcbline[1])
    for i in range(len(component)):
        plt.text(component[i].x, component[i].y, component[i].instance_name,fontsize=10)
    for i in range(len(pin)):
        for j in range(len(pin[i])):
            if j == 0: continue
            x = pin[i][j][3]
            y = pin[i][j][4]
            for k in range(len(padstack)):
                if padstack[k][0] == pin[i][j][1] and padstack[k][2] == '(rect':
                    plt.scatter(x,y,s=5)
                    plt.plot([x+padstack[k][4],x+padstack[k][4],x+padstack[k][6],x+padstack[k][6],x+padstack[k][4]],[y+padstack[k][5],y+padstack[k][7],y+padstack[k][7],y+padstack[k][5],y+padstack[k][5]], marker=',')
                if padstack[k][0] == pin[i][j][1] and padstack[k][2] == '(circle':
                      plt.scatter(x,y,s=int(padstack[k][4]/2))
    for i in range(len(result)):
        for j in range(len(result[i])):
            for k in range(len(result[i][j])):
                tmp  = result[i][j][k].split(' ')
                for l in range(len(tmp)):
                    tmp[l] = int(tmp[l])
                topx.append([])
                topy.append([])
                topx[len(topx)-1].append(tmp[0])
                topx[len(topx)-1].append(tmp[2])
                topy[len(topy)-1].append(tmp[1])
                topy[len(topy)-1].append(tmp[3])
    for i in range(len(topx)):
        plt.plot(topx[i], topy[i], marker=',')

    plt.show() 
# =============================================================
def notpath():
    for i in range(len(pin)):
        for j in range(len(pin[i])):
            if j == 0: continue
            pad = pin[i][j][1]
            lefttop = leftdown = righttop = rightdown = ()
            pad_num = 0
            for k in range(len(padstack)):
                if pad == padstack[k][0]:
                    if padstack[k][2] == '(rect':
                        pad_num = k
                        lefttop = (pin[i][j][3]+padstack[k][4],pin[i][j][4]+padstack[k][7])
                        leftdown = (pin[i][j][3]+padstack[k][4],pin[i][j][4]+padstack[k][5])
                        righttop = (pin[i][j][3]+padstack[k][6],pin[i][j][4]+padstack[k][7])
                        rightdown = (pin[i][j][3]+padstack[k][6],pin[i][j][4]+padstack[k][5])
                    break
            if padstack[pad_num][2] == '(rect': #(92, 126)   (92, 118)   (100, 126)   (100, 118)
                for m in range(lefttop[0],righttop[0]+1):
                    for n in range(leftdown[1],lefttop[1]+1):
                        #print(m," ",n)
                        F[m,n,0] = 0
def ispath(paths, dir):
    start = dir[paths[0]][2]
    end = dir[paths[1]][2]
    for i in range(len(pin)):
        for j in range(len(pin[i])):
            if j == 0: continue
            if pin[i][j][2] == start or pin[i][j][2] == end:
                pad = pin[i][j][1]
                lefttop = leftdown = righttop = rightdown = ()
                pad_num = 0
                for k in range(len(padstack)):
                    if pad == padstack[k][0]:
                        if padstack[k][2] == '(rect':
                            pad_num = k
                            lefttop = (pin[i][j][3]+padstack[k][4],pin[i][j][4]+padstack[k][7])
                            leftdown = (pin[i][j][3]+padstack[k][4],pin[i][j][4]+padstack[k][5])
                            righttop = (pin[i][j][3]+padstack[k][6],pin[i][j][4]+padstack[k][7])
                            rightdown = (pin[i][j][3]+padstack[k][6],pin[i][j][4]+padstack[k][5])
                        break
                if padstack[pad_num][2] == '(rect':
                    for m in range(lefttop[0],righttop[0]+1):
                        for n in range(leftdown[1],lefttop[1]+1):
                            F[m,n,0] = 1
# =============================================================
def line_to_line(pathx, pathy):
    net = []
    path_start = (pathx[0],pathy[0])
    path_next = (pathx[1],pathy[1])
    path_end = ()
    tmp = 0 

    while True:
        k = tmp
        A = path_next[1] - path_start[1]
        B = path_start[0] - path_next[0]
        C = path_next[0]*path_start[1] - path_start[0]*path_next[1]
        for i in range(k,len(pathx)):
            distance = abs(A*pathx[i]+B*pathy[i]+C)/math.sqrt(A*A+B*B)
            if (distance != 0.0) or (i == len(pathx)-1):
                tmp = i - 1
                if i == len(pathx)-1:
                    path_end = (pathx[i],pathy[i])
                else:
                    path_end = (pathx[tmp],pathy[tmp])
                break
        line = str(path_start[0])+" "+str(path_start[1])+" "+str(path_end[0])+" "+str(path_end[1])
        net.append(line)
        path_start = (pathx[tmp],pathy[tmp])
        path_next = (pathx[tmp+1],pathy[tmp+1])
        if (tmp+1 == len(pathx)-1):
             break
    return net

# =============================================================
def main():
    
    read()
    
    np.random.seed(0)
    trans = {} #每個座標轉成字典
    count = -1
    gamma = 0.5
    lrn_rate = 0.5
    
    for i in range(0,environment_x):
        for j in range(0,environment_y):
            F[i,j,1] = -1
    for i in range(len(pin)):
        for j in range(len(pin[i])):
            if j == 0: continue
            F[pin[i][j][3],pin[i][j][4],0] = 0
            F[pin[i][j][3],pin[i][j][4],1] = get_key(pin_dir,(pin[i][j][2]))
    for i in range(0,environment_x):
        for j in range(0,environment_y):
            count = count + 1
            trans[count] = (i,j)           
# =============================================================
    print(environment_x," ",environment_y)
    
    for i in range(len(net)):
        dir = []
        c = -1
        count = 0
        lay = 0
        for j in range(len(net[i])): #find the two pins shortest distance 
            if j == 0:
                continue
            else:
                c = c + 1
                dir.append([])
                for k in range(len(pin)):
                    for m in range(len(pin[k])):
                        if net[i][j] == pin[k][m][2]:
                            dir[c].append(pin[k][m][3]) #x
                            dir[c].append(pin[k][m][4]) #y
                            dir[c].append(pin[k][m][2]) #pin name
                            break
        #print("dir: ",dir)
        paths = point_connect(dir)
        for j in range(len(paths)):
            max_epochs = 1000
            maze = np.full((len(trans),8),0) #只存每個點可以到的8個方位   
            key_val = []
            Q = np.zeros(shape=[len(trans),8], dtype=np.float32)  # Quality
            time_s=time.time()
            #除了起終點，其他pad範圍不可走
            notpath()
            #把起終點設為可走
            ispath(paths[j], dir)
            F[dir[paths[j][0]][0],dir[paths[j][0]][1],0] = 1
            F[dir[paths[j][1]][0],dir[paths[j][1]][1],0] = 1
            print("(",dir[paths[j][0]][0],",",dir[paths[j][0]][1],") to (",dir[paths[j][1]][0],",",dir[paths[j][1]][1],")")
            count = -1
            for m in tqdm(range(len(trans))):
                tmp = trans[m] #(x,y)
                count = count + 1
                key_val.append([])
                for n in range(len(actions)):
                    if tmp[1] < environment_y - 1 and n == 0 and F[tmp[0],tmp[1]+1,0] != 0: #top
                        idx = tmp[0]*environment_y + (tmp[1]+1)
                        t = []
                        t.append(0)
                        t.append(idx)
                        key_val[count].append(t)
                        maze[m,0] = 1
                    elif tmp[1] > 0 and n == 1 and F[tmp[0],tmp[1]-1,0] != 0: #down
                        idx = tmp[0]*environment_y + (tmp[1]-1)
                        t = []
                        t.append(1)
                        t.append(idx)
                        key_val[count].append(t)
                        maze[m,1] = 1
                    elif tmp[0] > 0 and n == 2 and F[tmp[0]-1,tmp[1],0] != 0: #left
                        idx = (tmp[0]-1)*environment_y + tmp[1]
                        t = []
                        t.append(2)
                        t.append(idx)
                        key_val[count].append(t)
                        maze[m,2] = 1
                    elif tmp[0] < environment_x - 1 and n == 3 and F[tmp[0]+1,tmp[1],0] != 0: #right
                        idx = (tmp[0]+1)*environment_y + tmp[1]
                        t = []
                        t.append(3)
                        t.append(idx)
                        key_val[count].append(t)
                        maze[m,3] = 1
                    elif tmp[0] > 0 and tmp[1] < environment_y - 1 and n == 4 and F[tmp[0]-1,tmp[1]+1,0] != 0: #lefttop
                        idx = (tmp[0]-1)*environment_y + (tmp[1]+1)
                        t = []
                        t.append(4)
                        t.append(idx)
                        key_val[count].append(t)
                        maze[m,4] = 1
                    elif tmp[0] > 0 and tmp[1] > 0 and n == 5 and F[tmp[0]-1,tmp[1]-1,0] != 0: #leftdown
                        idx = (tmp[0]-1)*environment_y + (tmp[1]-1)
                        t = []
                        t.append(5)
                        t.append(idx)
                        key_val[count].append(t)
                        maze[m,5] = 1
                    elif tmp[0] < environment_x - 1 and tmp[1] < environment_y - 1 and n == 6 and F[tmp[0]+1,tmp[1]+1,0] != 0: #righttop
                        idx = (tmp[0]+1)*environment_y + (tmp[1]+1)
                        t = []
                        t.append(6)
                        t.append(idx)
                        key_val[count].append(t)
                        maze[m,6] = 1
                    elif tmp[0] < environment_x - 1 and tmp[1] > 0 and n == 7 and F[tmp[0]+1,tmp[1]-1,0] != 0: #rightdown
                        idx = (tmp[0]+1)*environment_y + (tmp[1]-1)
                        t = []
                        t.append(7)
                        t.append(idx)
                        key_val[count].append(t)
                        maze[m,7] = 1
            R = np.full((len(trans),8),0 , dtype=np.int) 
            for m in range(len(maze)):
                for n in range(0,8):
                    if maze[m,n] != 0:
                        R[m,n] = -0.1
            start = dir[paths[j][0]][0]*environment_y+dir[paths[j][0]][1]
            goal = dir[paths[j][1]][0]*environment_y+dir[paths[j][1]][1]
            print("start ",start," goal ",goal)
            for j in range(0,8):
                if maze[goal,j] == 1:
                    R[goal,j] = 10.0
            tran_time_start=time.time()
            epochs = 1000
            while max_epochs != 3500:
                train(maze, R, Q, gamma, lrn_rate, start, goal, trans, epochs, key_val)
                print("epochs: ",epochs)
                pathx,pathy = walk(start, goal, Q, trans, key_val)
                if pathx == None or pathy == None:
                    epochs = epochs + 500
                    max_epochs = epochs
                    continue
                else: break
            tran_time_end=time.time()
            print('tran time cost',tran_time_end-tran_time_start,'s')
            if pathx != None:
                for k in range(len(pathx)):
                    F[pathx[k],pathy[k]] = 0 
    time_e=time.time()
    print('time cost',time_e-time_s,'s')
    print("======================================================") 
if __name__ == "__main__":
    main()
    time_end=time.time()
    print('time cost',time_end-time_start,'s')